## Kelley hu4d5-5 VL-T31A error analysis

In this notebook we will use a random forest model to find the most energetically influential degrees of freedom for the VL-T31A TI production run. Next, we will compare the sampling of these DOF during TI production to a free energy profile derived from end state GaMD sampling. We will attempt to  correct any inaccurate sampling in the TI data and find the estimated ddG before and after the correction. 

In [1]:
from common_functions import *

### Ingesting initial TI data (bound state)

In [2]:
os.chdir("./TI_data/VL-T31A")
geom_dvdls = pd.read_csv("T31A_bound.csv")
geom_dvdls_ub = pd.read_csv("T31A_unbound.csv")

### Computing original TI estimate and error

In [3]:
orig_dG_bd = geom_dvdls.groupby("Lambda").mean().sum()["weight_dvdl"]
orig_dG_ubd = geom_dvdls_ub.groupby("Lambda").mean().sum()["weight_dvdl"]

empirical_value = 0.80
orig_error = abs((orig_dG_bd - orig_dG_ubd) - empirical_value)

print("Original ddG estimate: ")
print(f"{round(orig_dG_bd - orig_dG_ubd, 4)} kcal/mol ")
print()
print("Original ddG error: ")
print(f"{round(orig_error, 4)} kcal/mol")

Original ddG estimate: 
-0.1525 kcal/mol 

Original ddG error: 
0.9525 kcal/mol


### Removing correlated variables among candidate degrees of freedom

This is important because we want to limit the noise in our model training. See our methods/supplemental methods section for our process to choose the input features.

In [8]:
X = geom_dvdls.drop([
    "#Frame", "weight_dvdl", "dvdl", "Run", "Lambda", 
], axis=1)

Y = geom_dvdls["weight_dvdl"]

X_scl = pd.DataFrame(StandardScaler().fit_transform(X))
X_scl.columns = X.columns


### Checking to see if there is any cross-correlations within the independent variables

In [9]:
absCorr = abs(X_scl.corr())
for i in absCorr.columns:
    for j in absCorr.index:
        cor = absCorr.loc[i, j]
        if abs(cor) > 0.5 and i != j:
            print(i, j)
            print(cor)
            

### Using random forest model to identify the most energetically influential degrees of freedom

We run our model 25 times, then sort the results by the mean of feature importance across the 25 iterations. For this particular perturbation, the model $R^2$ between the geometric DOF (nearby side chain rotamers or interatomic distances) and the energetic DV/DL was not strong enough for us to check the sampling of these degrees of freedom. 

In [10]:
rfeDefault = RFE(estimator=DecisionTreeRegressor(max_depth=5, random_state=42), n_features_to_select=0.75, step=0.05)
rfDefault = RandomForestRegressor(
    max_depth=10, n_estimators=200, oob_score=True, max_features=0.6, min_samples_leaf = 7, min_samples_split=14, random_state=42
)

pipelineDefault_rf = Pipeline([
    ('feature_scaling', StandardScaler()),
    ('feature_selection', rfeDefault),
    ('regression_model', rfDefault)
])


imps = benchmark_model(pipelineDefault_rf, X_scl, Y, geom_dvdls["Lambda"])
imps[["Mean", "Median"]].sort_values(by="Mean", ascending=False)[:15]

Avg. training r2: 
0.5672
Training r2 std dev: 
0.0009
Avg. test r2: 
0.4018
Testing r2 std dev: 
0.0052


,Mean,Median
R421_chi3,0.152591,0.190558
N385_T386,0.117374,0.115940
N385_chi2,0.078927,0.089095
R421_chi2,0.073022,0.054773
R421_chi4,0.067216,0.063308
Q115_T386,0.062009,0.062003
Q115_chi1,0.057774,0.057173
R421_chi1,0.052132,0.055499
H446_chi1,0.051460,0.051589
S407_chi1,0.045580,0.050087
